In [1]:
from src.models.lightningmodel import LightningClassifierModelWrapper, modelling_choice, count_parameters
from src.preprocessing.data_loader import GeoEye1, compute_dataset_statistics, get_transforms, get_dataloaders
import torch
import pandas as pd

In [2]:
root_dir="ipeo_hurricane_for_students"
trainer, lightning_model = modelling_choice(model_name="resnet18", max_epochs=40, pretrained=True)
#mean, std = compute_dataset_statistics(root_dir, split="train", batch_size=1000)

Seed set to 42
/home/nstaehel/.venv/lib64/python3.9/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/nstaehel/.venv/lib64/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/nstaehel/.venv/lib64/python3.9/site-packages/i ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [3]:
mean = torch.load("src/preprocessing/mean.pt")
std = torch.load("src/preprocessing/std.pt")

In [ ]:
train_loader, val_loader, test_loader = get_dataloaders(root_dir, mean=mean, std=std, batch_size=100)
checkpoint_path = "checkpoints/resnet18-epoch=4-val_accuracy=0.99-val_f1=0.50.ckpt"

import traceback
try:
    rs = trainer.predict(lightning_model, dataloaders=test_loader, ckpt_path=checkpoint_path)
except Exception as e:
    traceback.print_exc()
    print(type(e), repr(e))

y_hat, y = list(map(list, zip(*rs)))
y_hat = torch.vstack(y_hat)
y = torch.hstack(y)
accuracy = (y_hat.argmax(1) == y).float().mean()
print(f"test accuracy {accuracy:.4f}")
trainable_params, total_params = count_parameters(lightning_model)
print(f"Trainable parameters: {trainable_params}, Total parameters: {total_params}")    

Loaded 19000 images for train split
Loaded 2000 images for validation split
Loaded 2000 images for test split


Restoring states from the checkpoint path at checkpoints/resnet18-epoch=4-val_accuracy=0.99-val_f1=0.50.ckpt
Loaded model weights from the checkpoint at checkpoints/resnet18-epoch=4-val_accuracy=0.99-val_f1=0.50.ckpt
/home/nstaehel/.venv/lib64/python3.9/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/nstaehel/.venv/lib64/python3.9/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Predicting: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


AttributeError: 'tuple' object has no attribute 'tb_frame'